In [ ]:
import numpy as np
import cvxpy as cp
import numpy.linalg as la
import matplotlib.pyplot as plt
from IPython import display
import casadi as ca

from agents import DT_Kin_Bike_Agent
from LTV_FTOCP import LTV_FTOCP
from NL_FTOCP import NL_FTOCP

In [ ]:
# model_dt = 0.01
# control_dt = 0.05

model_dt = 0.1
control_dt = 0.1

n_control = int(np.around(control_dt/model_dt))

x_0 = np.array([0.0, -6.0, 0.0, 0.0])
x_f = np.array([0.0, 6.0, 3.0*np.pi/4.0, 0.0])

waypts = [np.array([5.0, 0.0, np.pi/2.0, 1.0])]
# waypts = []
waypts.append(x_f)

l_r = 0.5
l_f = 0.5
w = 0.5

Q = np.diag([15.0, 15.0, 20.0, 25.0])
R = np.diag([1.0, 10.0])
Rd = 0.1*np.eye(2)
P = Q
N = 15

dyn_agent = DT_Kin_Bike_Agent(l_r, l_f, w, model_dt, x_0)
ctr_agent = DT_Kin_Bike_Agent(l_r, l_f, w, control_dt, x_0)

# fhocp = InitFTOCP(Q, P, R, Rd, N, dyn_agent, x_refs=waypts)
ftocp = LTV_FTOCP(Q, P, R, Rd, N, ctr_agent, x_refs=waypts)
nl_ftocp = NL_FTOCP(N, ctr_agent)
waypt_idx = ftocp.get_reference_idx()

In [ ]:
# LTV MPC
x_traj = [x_0]
u_traj = []

x_t = np.squeeze(x_0)
t = 0
t_span = [0]

counter = 0

# for i in range(1000):
while True:
#     print(i)
    
    t_span.append(t)
    
    if np.mod(counter, n_control) == 0:
        x_pred, u_pred = ftocp.solve(x_t, t, verbose=False)
        u_t = u_pred[:,0]
        print('t: %g, d: %g, x: %g, y: %g, phi: %g, v: %g' % (t, la.norm(x_t[:2] - waypts[waypt_idx][:2]), x_t[0], x_t[1], x_t[2]*180.0/np.pi, x_t[3]))
#         print(x_pred)
    
    x_tp1 = dyn_agent.sim(x_t, u_t)
    
    x_traj.append(x_tp1)
    u_traj.append(u_t)
    
    d = la.norm(x_tp1[:2] - waypts[waypt_idx][:2])
    v = x_tp1[3] - waypts[waypt_idx][3]
    if d <= 0.5 and waypt_idx < len(waypts)-1:
        print('Waypoint %i reached' % waypt_idx)
        ftocp.advance_reference_idx()
        waypt_idx = ftocp.get_reference_idx()
    elif d <= 0.5 and v <= 0.1 and waypt_idx == len(waypts)-1:
        print('Goal state reached')
        break
            
    t += model_dt
    counter += 1
    x_t = x_tp1

x_traj = np.array(x_traj)
u_traj = np.array(u_traj)

In [ ]:
# NL MPC
x_f = np.array([3.0, -3.0, 3.0*np.pi/4.0, 0.0])

x_traj_nl = [x_0]
u_traj_nl = []

x_t = np.squeeze(x_0)
t = 0
t_span = [0]

counter = 0

# for i in range(1000):
while True:
#     print(i)
    
    t_span.append(t)
    
    if np.mod(counter, n_control) == 0:
        x_pred, u_pred = nl_ftocp.solve(x_t, t, x_f, verbose=False)
        u_t = u_pred[:,0]
        print('t: %g, d: %g, x: %g, y: %g, phi: %g, v: %g' % (t, la.norm(x_t[:2] - waypts[waypt_idx][:2]), x_t[0], x_t[1], x_t[2]*180.0/np.pi, x_t[3]))
#         print(x_pred)
    
    x_tp1 = dyn_agent.sim(x_t, u_t)
    
    x_traj_nl.append(x_tp1)
    u_traj_nl.append(u_t)
    
    d = la.norm(x_tp1[:2] - x_f[:2])
    v = x_tp1[3] - x_f[3]
    if d <= 0.5 and v <= 0.1:
        print('Goal state reached')
        break
            
    t += model_dt
    counter += 1
    x_t = x_tp1

x_traj_nl = np.array(x_traj_nl)
u_traj_nl = np.array(u_traj_nl)

In [ ]:
fig = plt.figure()

xy_ax = fig.add_axes([0, 0, 1, 1])
psi_ax = fig.add_axes([1.1, 0.9, 1, 0.2])
psi_ax.set_xticks([])
v_ax = fig.add_axes([1.1, 0.6, 1, 0.2])
v_ax.set_xticks([])
df_ax = fig.add_axes([1.1, 0.3, 1, 0.2])
df_ax.set_xticks([])
a_ax = fig.add_axes([1.1, 0.0, 1, 0.2])

xy_ax.plot(x_traj[:,0], x_traj[:,1], 'b-')
xy_ax.set_xlabel('x')
xy_ax.set_ylabel('y')
xy_ax.set_xlim([-10, 10])
xy_ax.set_ylim([-10, 10])
xy_ax.set_aspect('equal')    
for j in range(len(waypts)):
    xy_ax.plot(waypts[j][0], waypts[j][1], 'ro')

psi_ax.plot(t_span, x_traj[:,2], 'b-')
psi_ax.set_ylabel('psi')

v_ax.plot(t_span, x_traj[:,3], 'b-')
v_ax.set_ylabel('v')

df_ax.plot(t_span[:-1], u_traj[:,0], 'b-')
df_ax.set_ylabel('df')

a_ax.plot(t_span[:-1], u_traj[:,1], 'b-')
a_ax.set_ylabel('a')

In [ ]:
f = plt.figure()
ax = f.add_axes([0, 0, 2, 2])
control_counter = -1

for i in range(0,u_traj.shape[0],n_control):
#     print(i)
    
    ax.clear()
    
    x_t = x_traj[i,:]
    u_t = u_traj[i,:]
    
    car_x = [x_t[0] + l_f*np.cos(x_t[2]) + w*np.sin(x_t[2])/2, 
            x_t[0] + l_f*np.cos(x_t[2]) - w*np.sin(x_t[2])/2,
            x_t[0] - l_r*np.cos(x_t[2]) - w*np.sin(x_t[2])/2,
            x_t[0] - l_r*np.cos(x_t[2]) + w*np.sin(x_t[2])/2,
            x_t[0] + l_f*np.cos(x_t[2]) + w*np.sin(x_t[2])/2]
    car_y = [x_t[1] + l_f*np.sin(x_t[2]) - w*np.cos(x_t[2])/2, 
            x_t[1] + l_f*np.sin(x_t[2]) + w*np.cos(x_t[2])/2,
            x_t[1] - l_r*np.sin(x_t[2]) + w*np.cos(x_t[2])/2,
            x_t[1] - l_r*np.sin(x_t[2]) - w*np.cos(x_t[2])/2,
            x_t[1] + l_f*np.sin(x_t[2]) - w*np.cos(x_t[2])/2]
    
    wheel_x = [x_t[0] + l_f*np.cos(x_t[2]) + 0.2*np.cos(x_t[2]+u_t[0]), x_t[0] + l_f*np.cos(x_t[2]) - 0.2*np.cos(x_t[2]+u_t[0])]
    wheel_y = [x_t[1] + l_f*np.sin(x_t[2]) + 0.2*np.sin(x_t[2]+u_t[0]), x_t[1] + l_f*np.sin(x_t[2]) - 0.2*np.sin(x_t[2]+u_t[0])]
    ax.plot(car_x, car_y)
    ax.plot(wheel_x, wheel_y)
    
    for j in range(len(waypts)):
        ax.plot(waypts[j][0], waypts[j][1], 'ro')
    
    if np.mod(i, n_control) == 0:
        control_counter += 1
    ax.plot(ftocp.x_preds[0,:,control_counter], ftocp.x_preds[1,:,control_counter], 'k.-')
    
    ax.set_xlim([-10, 10])
    ax.set_ylim([-10, 10])
    ax.set_aspect('equal')
    plt.draw()
    
    display.clear_output(wait=True)
    display.display(f)
    

In [ ]:
A = np.array([[ 1.        ,  0.        , -0.3945411 , -0.00567603],
       [ 0.        ,  1.        , -0.04507999,  0.04967678],
       [ 0.        ,  0.        ,  1.        ,  0.00244248],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])
print(la.eig(A))

In [ ]:
A = np.random.randn(4,4)
B = np.random.randn(4,2)
c = np.random.randn(4)

A_pows = [la.matrix_power(A, i) for i in range(10+1)]
A_pows = np.concatenate(A_pows, axis=1)
B_mat = np.tile(B, (10,1))
c_vec = np.tile(c, 10)

print(A_pows.shape)
print(B_mat.shape)
print(c_vec.shape)

print(A_pows[:,-4:], A_pows[:,:4])
print(A_pows[:,:-4].dot(c_vec).shape)

In [ ]:
opti = ca.Opti()
opti.minimize(1)

solver_opts = {
            "mu_strategy" : "adaptive",
            "mu_init" : 1e-5,
            "mu_min" : 1e-15,
            "barrier_tol_factor" : 1,
            "print_level" : 5,
            "linear_solver" : "ma27"
            }
plugin_opts = {"verbose" : False, "print_time" : False, "print_out" : False}

opti.solver('ipopt', plugin_opts, solver_opts)

sol = opti.solve()